In [1]:
#pragma cling add_include_path("../../libtorch/include")
#pragma cling add_include_path("../../libtorch/include/torch/csrc/api/include")
#pragma cling add_library_path("../../libtorch/lib")
#pragma cling load("libtorch")

In [2]:
#include <iostream>
#include <vector>
#include <torch/torch.h>

# 1 create tensor by picking a factory function in libtorch


## 1.1 using factory function

In [3]:
/* Factory Functions
    A factory function is a function that produces a new tensor. There are many factory functions available in PyTorch (both in Python and C++), which differ in the way they initialize a new tensor before returning it. All factory functions adhere to the following general “schema”:
    torch::<function-name>(<function-specific-options>, <sizes>, <tensor-options>)
*/

In [4]:
torch::Tensor tensor =  torch::rand({2,3});
std::cout << tensor << std::endl;

// <sizes> is an object of type IntArrayRef and specifies the shape of the resulting tensor,
std::cout << "tensor size: " << tensor.sizes() << std::endl;

 0.8808  0.8306  0.6456
 0.2943  0.2204  0.3642
[ CPUFloatType{2,3} ]
tensor size: [2, 3]


In [10]:
std::cout << tensor.sizes()[0];

2

In [11]:
std::cout << tensor.size(0);

2

In [4]:
// 1.1.2 Passing Function-Specific Parameters
torch::Tensor tensor_int = torch::randint(/*high=*/10, {2, 3});
std::cout << tensor_int << std::endl;

 9  0  7
 2  3  2
[ CPUFloatType{2,3} ]


## 1.2 create tensor from existing data
* torch::from_blob function

* torch::tensor function

In [21]:
std::vector<float> data_vector1d = {1,2,3,4};
std::cout << "data vector 1d: \n" << data_vector1d << std::endl;

std::vector<std::vector<float>> data_vector2d = {{1,2,3,4},{2,3,4,5}};
std::cout << "data vector 2d: \n" << data_vector2d << std::endl;

data vector 1d: 
1 2 3 4
data vector 2d: 
1 2 3 4 2 3 4 5


In [30]:
torch::Tensor tensor_from_vector1d = torch::from_blob(data_vector1d.data(), {2,2});
std::cout << "tensor from vector1d: \n" << tensor_from_vector1d << std::endl;

tensor from vector1d: 
 1  2
 3  4
[ CPUFloatType{2,2} ]


In [31]:
// error
torch::Tensor tensor_from_vector2d = torch::from_blob(data_vector2d.data(), {2,4});
std::cout << "tensor from vector2d: \n" << tensor_from_vector2d << std::endl;

tensor from vector2d: 
 5.2036e-06  3.0634e-41  5.2036e-06  3.0634e-41
 5.2036e-06  3.0634e-41  5.4529e-06  3.0634e-41
[ CPUFloatType{2,4} ]


In [32]:
/*
torch::tensor create function

*/
torch::Tensor tensor_from_vector1d_ = torch::tensor(data_vector1d);
std::cout << "tensor from vector1d: \n" << tensor_from_vector1d << std::endl;

tensor from vector1d: 
 1  2
 3  4
[ CPUFloatType{2,2} ]


In [33]:
// error
//torch::Tensor tensor_from_vector2d = torch::tensor(data_vector2d);
//std::cout << "tensor from vector: \n" << tensor_from_vector2d << std::endl;

# 2 simple operation to tensor
* member function of tensor
* in-built function in libtorch

In [62]:
torch::Tensor a = torch::rand({2,3});

In [64]:
std::cout << a << std::endl;

 0.2044  0.0009  0.8855
 0.3952  0.8131  0.5710
[ CPUFloatType{2,3} ]


In [66]:
auto b = a.reshape({3,2});
std::cout << b << std::endl;

 0.2044  0.0009
 0.8855  0.3952
 0.8131  0.5710
[ CPUFloatType{3,2} ]


In [70]:
auto c = a.pow(2);
std::cout << c << std::endl;

 4.1769e-02  7.3036e-07  7.8416e-01
 1.5616e-01  6.6110e-01  3.2599e-01
[ CPUFloatType{2,3} ]


In [71]:
auto d = a.abs();
std::cout << d << std::endl;

 0.2044  0.0009  0.8855
 0.3952  0.8131  0.5710
[ CPUFloatType{2,3} ]


In [72]:
auto e = torch::abs(a);
std::cout << e << std::endl;

 0.2044  0.0009  0.8855
 0.3952  0.8131  0.5710
[ CPUFloatType{2,3} ]


# 3 explore properties of tensor

In [42]:
std::cout << tensor_from_vector1d.sizes() << std::endl;

[2, 2]


In [43]:
std::cout << tensor_from_vector1d.dtype() << std::endl;

float


In [44]:
std::cout << tensor_from_vector1d.device() << std::endl;

cpu


In [46]:
std::cout << tensor_from_vector1d.layout() << std::endl;

Strided


In [47]:
std::cout << tensor_from_vector1d.requires_grad() << std::endl;

0


# 4 configure properties of the tensor

The previous section discussed function-specific arguments. Function-specific arguments can only change the values with which tensors are filled, and sometimes the size of the tensor. They never change things like the data type (e.g. float32 or int64) of the tensor being created, or whether it lives in CPU or GPU memory. The specification of these properties is left to the very last argument to every factory function: a TensorOptions object, discussed below.
TensorOptions is a class that encapsulates the construction axes of a Tensor. With construction axis we mean a particular property of a Tensor that can be configured before its construction (and sometimes changed afterwards). These construction axes are:
The dtype (previously “scalar type”), which controls the data type of the elements stored in the tensor,
The layout, which is either strided (dense) or sparse,
The device, which represents a compute device on which a tensor is stored (like a CPU or CUDA GPU),
The requires_grad boolean to enable or disable gradient recording for a tensor,
If you are used to PyTorch in Python, these axes will sound very familiar. The allowed values for these axes at the moment are:
For dtype: kUInt8, kInt8, kInt16, kInt32, kInt64, kFloat32 and kFloat64,
For layout: kStrided and kSparse,
For device: Either kCPU, or kCUDA (which accepts an optional device index),
For requires_grad: either true or false.*/


## 4.1 use the general solution: torch::TensorOptions

In [49]:
 auto options = torch::TensorOptions()
	    .dtype(torch::kFloat32)
	    .layout(torch::kStrided)
	    .device(torch::kCPU)
	    .requires_grad(true);
torch::Tensor tensor_dtype = torch::full({3, 4}, /*value=*/12.3, options);
std::cout << "tensor from torch::full: \n" << tensor_dtype << std::endl;

assert(tensor_dtype.dtype() == torch::kFloat32);
assert(tensor_dtype.layout() == torch::kStrided);
assert(tensor_dtype.device().type() == torch::kCPU); // or device().is_cuda()
assert(tensor_dtype.requires_grad());

tensor from torch::full: 
 12.3000  12.3000  12.3000  12.3000
 12.3000  12.3000  12.3000  12.3000
 12.3000  12.3000  12.3000  12.3000
[ CPUFloatType{3,4} ]


## 4.2 equivalent operation regarding tensor property

 Now, you may be thinking: do I really need to specify each axis for every new tensor I create?
       Fortunately, the answer is “no”, as every axis has a default value. These defaults are:
       kFloat32 for the dtype,
       kStrided for the layout,
       kCPU for the device,
       false for requires_grad.
    What this means is that any axis you omit during the construction of a TensorOptions object will take on its default value. For example, this is our previous TensorOptions object, but with the dtype and layout defaulted:
    in fact, we can even omit all axes to get an entirely defaulted TensorOptions object:
    auto options = torch::TensorOptions(); // or `torch::TensorOptions options;`
    A nice consequence of this is that the TensorOptions object we just spoke so much about can be entirely omitted from any tensor factory call:
    A 32-bit float, strided, CPU tensor that does not require a gradient.
    torch::Tensor tensor = torch::randn({3, 4});
    torch::Tensor range = torch::arange(5, 10);


In [56]:
// equivalent operation
torch::ones(10, torch::TensorOptions().dtype(torch::kFloat32));
torch::ones(10, torch::dtype(torch::kFloat32));
//and further instead of
torch::ones(10, torch::TensorOptions().dtype(torch::kFloat32).layout(torch::kStrided));
//we can just write
torch::ones(10, torch::dtype(torch::kFloat32).layout(torch::kStrided));
// use default property
// python syntax: torch.randn(3, 4, dtype=torch.float32, device=torch.device('cuda', 1), requires_grad=True)

@0x55653829be20

## 4.3 properties conversion


Just as we can use TensorOptions to configure how new tensors should be created, we can also use TensorOptions to convert a tensor from one set of properties to a new set of properties. Such a conversion usually creates a new tensor and does not occur in-place. For example, if we have a source_tensor created with


In [55]:
torch::Tensor source_tensor = torch::randn({2, 3});

//we can convert it from int64 to float32:
torch::Tensor float_tensor = source_tensor.to(torch::kFloat32);

// The result of the conversion, float_tensor, is a new tensor pointing to new memory, unrelated to the source source_tensor.
// We can then move it from CPU memory to GPU memory:
    // if (torch::cuda::is_available()){
    //    torch::Tensor gpu_tensor = float_tensor.to(torch::kCUDA);
    //}